# MCDonals Dataset (restaurant) 5k

In [1]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

from gensim.models.word2vec import Word2Vec
from gensim.parsing.preprocessing import (
    strip_punctuation,
    strip_numeric,
    strip_short,
    stem_text,
    strip_multiple_whitespaces,
    remove_stopwords,
    STOPWORDS,
)



from bs4 import BeautifulSoup
from collections import defaultdict
#import networkx as nx
import matplotlib.pyplot as plt
#from pyvis.network import Network
import time

# ejecutar dataframe de forma paralela
from pandarallel import pandarallel  # import pandarallel

pandarallel.initialize()  # initialize pandarallel

[nltk_data] Downloading package wordnet to /home/ymamani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ymamani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ymamani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
path_mcdonals_file = "mcdonald_reviews_5k.csv"
ds_mcdonals_reviews = pd.read_csv(path_mcdonals_file, encoding="latin-1")

In [3]:
# ds_mcdonals_reviews = ds_mcdonals_reviews.drop(
#     [
#         "reviewer_id",
#         "store_name",
#         "category",
#         "store_address",
#         #'latitude',
#         "longitude",
#         "rating_count",
#         "review_time",
#     ],
#     axis=1,
# )

In [4]:
# ds_mcdonals_reviews = ds_mcdonals_reviews[["review", "rating"]]

In [3]:
#reemplazando los valores de rating con numeros
replacements = {"1 star": 1, "2 stars": 2, "3 stars": 3, "4 stars":4, "5 stars":5}
ds_mcdonals_reviews.replace(replacements, inplace=True)

/tmp/ipykernel_2909673/794213423.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds_mcdonals_reviews.replace(replacements, inplace=True)


In [4]:
# extender contracciones
contractions_dict = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
}

In [5]:
# Función para extender contracciones
def expand_contractions(text):
    text = str(text).lower()
    for contraction, expansion in contractions_dict.items():
        text = text.replace(contraction, expansion)
    return text


# aplicando funcion expand_contractions
ds_mcdonals_reviews["contraction_review"] = ds_mcdonals_reviews[
    "review"
].parallel_apply(expand_contractions)

In [6]:
# funcion para limpieza de textos

def clean_text(text):
    text = str(text).lower()
    #text = BeautifulSoup(text, "lxml").getText()  # remove etiquetas html
    text = re.sub(
        r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE
    )  # Elimina URLs    
    text = re.sub(
        r"\@\w+|\#\w+", "", text
    )  # Elimina las menciones @ y '#' de las redes sociales
    text = re.sub(r" +", " ", text)  # elimina espacios en blanco
    text = strip_punctuation(text)  # Elimina los caracteres de puntuación
    text = strip_numeric(text)  # Elimina los números
    text = re.sub("[^a-zA-Z0-9\s]+","",text)
    text = strip_multiple_whitespaces(text)
    # text = strip_short(text,minsize=2)# Elimina las palabras cortas
    text = re.sub(
        r"\b\w{20,}\b", "", text
    )  # elimina palabras largar de 20 caracteres a mas.
    # text = set(stopwords.words('english'))
    # text = remove_stopwords(text)
    return text


ds_mcdonals_reviews["clean_review"] = ds_mcdonals_reviews[
    "contraction_review"
].parallel_apply(clean_text)

In [7]:
def remove_stopwords_text(text):
    text = text.split()
    sw_text = stopwords.words("english")
    sw_text = [word for word in text if not word in sw_text]
    final_sw_text = " ".join(sw_text)
    return final_sw_text


ds_mcdonals_reviews["sw_review"] = ds_mcdonals_reviews["clean_review"].parallel_apply(
    remove_stopwords_text
)

In [8]:
ds_mcdonals_reviews = ds_mcdonals_reviews.drop(
    columns=["review", "contraction_review", "clean_review"]
)
ds_mcdonals_reviews["polarity"] = ds_mcdonals_reviews["rating"].parallel_apply(
    lambda x: 1 if x >= 4 else 0
)

In [9]:
ds_mcdonals_reviews["polarity"].value_counts()

polarity
1    2693
0    2307
Name: count, dtype: int64

In [10]:
import gensim
emb_wn_graph = gensim.models.KeyedVectors.load_word2vec_format('graph_model_embedding_wn_w5d300_le40nw300a09_2k.txt', binary=False)
emb_cn_graph = gensim.models.KeyedVectors.load_word2vec_format('graph_model_embedding_cn_w5d300_le40nw300a09_2k.txt', binary=False)



In [11]:
# Tokenización y representación de documentos
def document_vector(doc, model):
    words = doc.split()
    vectors = [
        model[word] for word in words if word in model
    ]  # vectors = [embeddings.get(word, np.zeros(300)) for word in words]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # return np.zeros(model.vector_size)  # Vector de ceros si no se encuentran palabras
        return np.zeros(300)

In [12]:
# Aplicar tokenización y representación a todas las reseñas
ds_mcdonals_reviews["vector_sentence_wn_g_sg"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, emb_wn_graph))
ds_mcdonals_reviews["vector_sentence_cn_g_sg"] = ds_mcdonals_reviews["sw_review"].parallel_apply(lambda x: document_vector(x, emb_cn_graph))



In [13]:
ds_mcdonals_reviews_wn = ds_mcdonals_reviews.drop(columns=["rating", "sw_review", "vector_sentence_cn_g_sg"])
ds_mcdonals_reviews_cn = ds_mcdonals_reviews.drop(columns=["rating", "sw_review", "vector_sentence_wn_g_sg"])

In [14]:
x_ds_mcdonals_reviews_wn = np.vstack(ds_mcdonals_reviews_wn["vector_sentence_wn_g_sg"])
y_ds_mcdonals_reviews_wn = ds_mcdonals_reviews_wn["polarity"]

x_ds_mcdonals_reviews_cn = np.vstack(ds_mcdonals_reviews_cn["vector_sentence_cn_g_sg"])
y_ds_mcdonals_reviews_cn = ds_mcdonals_reviews_cn["polarity"]

In [15]:
from sklearn.model_selection import train_test_split, cross_validate
x_train_wn, x_test_wn, y_train_wn, y_test_wn = train_test_split(
    x_ds_mcdonals_reviews_wn,
    y_ds_mcdonals_reviews_wn,
    test_size=0.2,
    random_state=42,
)

x_train_cn, x_test_cn, y_train_cn, y_test_cn = train_test_split(
    x_ds_mcdonals_reviews_cn,
    y_ds_mcdonals_reviews_cn,
    test_size=0.2,
    random_state=42,
)

In [16]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import (
    accuracy_score,
    accuracy_score,
    roc_auc_score,
    f1_score,
    classification_report,
    precision_score,
    recall_score,
)

In [17]:
def Evaluation(model, X_train, X_test, y_train, y_test, hypertuning=False):
    
    y_pred = model.predict(X_train)
    y_pred_proba = model.predict_proba(X_train)

    accuracy_train = accuracy_score(y_train, y_pred)
    precision_train = precision_score(y_train, y_pred)
    recall_train = recall_score(y_train, y_pred)
    F1_score_train = f1_score(y_train, y_pred)
    # print("F1_Score = ", F1_score_train )
    roc_auc_train = roc_auc_score(y_train, y_pred_proba[:, 1])
    # print( classification_report( y_train, y_pred ) )

    # print( " For Test Set :  ")
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy_test = accuracy_score(y_test, y_pred)
    precision_test = precision_score(y_test, y_pred)
    recall_test = recall_score(y_test, y_pred)
    F1_score_test = f1_score(y_test, y_pred)
    # print("F1_Score = ", F1_score_test )
    roc_auc_test = roc_auc_score(y_test, y_pred_proba[:, 1])
    
    #cross-validation
    cross_val = cross_val_score(model, X_train, y_train, cv=5).mean()
    
    return (
        accuracy_train,
        precision_train,
        recall_train,
        F1_score_train,
        roc_auc_train,
        accuracy_test,
        precision_test,
        recall_test,
        F1_score_test,
        roc_auc_test,
        cross_val
    )

In [18]:
def apply_models_with_default_paramters(X_train, X_test, y_train, y_test):
    models_default = [
        {"ModelNames": "SVM", "Model": SVC(kernel="poly", probability=True)},
        {
            "ModelNames": "RF",
            "Model": RandomForestClassifier(n_estimators=100, random_state=42),
        },
        {"ModelNames": "KNN", "Model": KNeighborsClassifier(n_neighbors=5)},
        {"ModelNames": "XGB", "Model": XGBClassifier()},
        {"ModelNames": "LR", "Model": LogisticRegression(max_iter=1000)},
    ]
    
    cross_val_train = []

    F1_Score_train = []
    Accuracy_train = []
    Recall_train = []
    Precision_train = []
    ROC_AUC_Score_train = []

    F1_Score_test = []
    Accuracy_test = []
    Recall_test = []
    Precision_test = []
    ROC_AUC_Score_test = []

    Model_Name = []

    for model in models_default:
        # print(model)
        Model_Name.append(model["ModelNames"])
        model["Model"].fit(X_train, y_train)

        (
            accuracy_train,
            precision_train,
            recall_train,
            F1_score_train,
            roc_auc_train,
            accuracy_test,
            precision_test,
            recall_test,
            F1_score_test,
            roc_auc_test,
            cross_val            
        ) = Evaluation(model["Model"], X_train, X_test, y_train, y_test, False)

        cross_val_train.append(cross_val)
        
        F1_Score_train.append(F1_score_train)
        Accuracy_train.append(accuracy_train)
        Recall_train.append(recall_train)
        Precision_train.append(precision_train)
        ROC_AUC_Score_train.append(roc_auc_train)

        F1_Score_test.append(F1_score_test)
        Accuracy_test.append(accuracy_test)
        Recall_test.append(recall_test)
        Precision_test.append(precision_test)
        ROC_AUC_Score_test.append(roc_auc_test)

    results = pd.DataFrame()
    results["Model_Name"] = Model_Name

    train_test_f1_score_difference = np.subtract(
        F1_Score_train, F1_Score_test
    )  # To Check Overfitting/Underfitting

    results["Cross validation mean"] = cross_val_train
    
    results["Accuracy on Test Set"] = Accuracy_test
    results["Precision on Test Set"] = Precision_test
    results["Recall on Test Set"] = Recall_test
    results["F1_Score on Test Set"] = F1_Score_test         
    results["ROC_AUC_Score on Test Set"] = ROC_AUC_Score_test

    results["Accuracy on Train Set"] = Accuracy_train
    results["Precision on Train Set"] = Precision_train
    results["Recall on Train Set"] = Recall_train
    results["F1_Score on Train Set"] = F1_Score_train      
    results["ROC_AUC_Score on Train Set"] = ROC_AUC_Score_train

    results["Difference of F1_Score on train and test"] = train_test_f1_score_difference

    results = results.sort_values(
        by=["F1_Score on Test Set", "Difference of F1_Score on train and test"],
        ascending=[False, True],
    )

    return results

In [19]:
def apply_model_plot(df_model):
    # Generar gráfico

    plt.figure(figsize=(8, 5))
    # plt.bar(models, accuracies, color=['orange', 'green', 'red'])
    bars = plt.bar(df_model['Model_Name'], df_model['Accuracy on Test Set'], color=["orange", "green", "red", "blue"])
    plt.ylim(0, 1.0)
    plt.xlabel("Modelo")
    plt.ylabel("Accuracy")
    plt.title("Accuracy test de Modelos de Sentimiento")
    for bar, accuracy in zip(bars, df_model['Accuracy on Test Set']):
        plt.text(
            bar.get_x() + bar.get_width() / 2 - 0.08,
            bar.get_height() + 0.02,
            f"{accuracy:.4f}",
            fontsize=12,
        )

    plt.show()

In [20]:
Results_wn_graph = apply_models_with_default_paramters(x_train_wn, x_test_wn, y_train_wn, y_test_wn)
Results_wn_graph

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
3,XGB,0.81200,0.837,0.876248,0.812963,0.843420,0.911926,0.97650,0.975081,0.981421,0.978241,0.997953,0.134821
4,LR,0.83175,0.838,0.884146,0.805556,0.843023,0.909344,0.86225,0.888835,0.850441,0.869214,0.938775,0.026191
1,RF,0.80900,0.815,0.861507,0.783333,0.820563,0.904775,0.97600,0.974182,0.981421,0.977788,0.992169,0.157226
2,KNN,0.79175,0.798,0.891204,0.712963,0.792181,0.871820,0.84775,0.910638,0.795170,0.848996,0.937795,0.056815
0,SVM,0.79025,0.799,0.922693,0.685185,0.786397,0.905252,0.84275,0.944574,0.751974,0.837342,0.949997,0.050944


In [20]:
Results_cn_graph = apply_models_with_default_paramters(x_train_cn, x_test_cn, y_train_cn, y_test_cn)
Results_cn_graph

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
3,XGB,0.82075,0.836,0.880567,0.805556,0.841393,0.916180,0.97850,0.979582,0.980492,0.980037,0.998242,0.138644
4,LR,0.82900,0.836,0.888430,0.796296,0.839844,0.915433,0.86800,0.891566,0.859266,0.875118,0.940935,0.035275
1,RF,0.80600,0.816,0.872385,0.772222,0.819253,0.903667,0.97800,0.975150,0.984208,0.979658,0.992466,0.160404
0,SVM,0.79025,0.798,0.928934,0.677778,0.783726,0.908663,0.83850,0.946651,0.741756,0.831771,0.952257,0.048045
2,KNN,0.77900,0.784,0.878505,0.696296,0.776860,0.866731,0.83625,0.910636,0.771482,0.835303,0.937409,0.058443
